# Spaceshooter Enviroment

## Import libiaries

In [ ]:
# Import libraries for SpaceShooter game
import pygame
import random
import time
import os

## Initialise settings 

In [ ]:
pygame.font.init()
pygame.mixer.init()

WIDTH, HEIGHT = 900, 500
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
BORDER = pygame.Rect(WIDTH//2 - 5, 0, 10, HEIGHT)
HEALTH_FONT = pygame.font.SysFont('comicsans', 40)
WINNER_FONT = pygame.font.SysFont('comicsans', 100)
FPS = 60

## Set colours

In [ ]:
BLACK = (0, 0, 0)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)
PURPLE = (200, 0, 200)

## Initialise physics


In [ ]:
VEL = 5
BULLET_VEL = 5
MAX_BULLETS = 4
SPACESHIP_WIDTH, SPACESHIP_HEIGHT = 55, 40

## Initialise images and sounds

In [ ]:
YELLOW_HIT = pygame.USEREVENT + 1
RED_HIT = pygame.USEREVENT + 2

# Heart image
HEART_IMAGE = pygame.image.load(os.path.join('Assets', 'heart.png')).convert_alpha()
HEART = pygame.transform.scale(HEART_IMAGE, (40, 40))

# Yellow spaceShip image
YELLOW_SPACESHIP_IMAGE = pygame.image.load(os.path.join('Assets', 'spaceship_yellow.png'))
YELLOW_SPACESHIP = pygame.transform.rotate(pygame.transform.scale(YELLOW_SPACESHIP_IMAGE, (SPACESHIP_WIDTH, SPACESHIP_HEIGHT)), 90)

# Red spaceship image
RED_SPACESHIP_IMAGE = pygame.image.load(os.path.join('Assets', 'spaceship_red.png'))
RED_SPACESHIP = pygame.transform.rotate(pygame.transform.scale(RED_SPACESHIP_IMAGE, (SPACESHIP_WIDTH, SPACESHIP_HEIGHT)), 270)

# Backround image
SPACE = pygame.transform.scale(pygame.image.load(os.path.join('Assets', 'space_background_2.jpg')), (WIDTH, HEIGHT))

# Game sounds
BULLET_HIT_SOUND = pygame.mixer.Sound('Assets/Grenade+1.mp3')
BULLET_FIRE_SOUND = pygame.mixer.Sound('Assets/Gun+Silencer.mp3')
EZPZ = pygame.mixer.Sound('Assets/ezpz.mp3')

## Build enviroment

In [ ]:
class SpaceShooterEnv:
    
    def __init__(self):
        # Init display
        pygame.display.set_caption("SpaceShooterEnv")
        self.BORDER = pygame.Rect(WIDTH//2 - 5, 0, 10, HEIGHT)
        self.clock = pygame.time.Clock()
        self.reset()
      
    def reset(self):
       # Reset game state
        self.red = pygame.Rect(700, 230, SPACESHIP_WIDTH, SPACESHIP_HEIGHT)
        self.yellow = pygame.Rect(100, 230, SPACESHIP_WIDTH, SPACESHIP_HEIGHT)
        self.red_bullets = []
        self.yellow_bullets = []
        self.red_health = 10
        self.yellow_health = 10  
        self.frame_iteration = 0    
        
    def step(self):
        self.frame_iteration += 1
        # Collect user input and perform action 
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
            
            if event.type == pygame.KEYDOWN:
                # Check if yellow spaceship has fired
                if event.key == pygame.K_LCTRL and len(self.yellow_bullets) < MAX_BULLETS:
                    self.bullet = pygame.Rect(self.yellow.x + self.yellow.width, self.yellow.y + self.yellow.height//2, 10, 5)
                    self.yellow_bullets.append(self.bullet)
                    BULLET_FIRE_SOUND.play()
            
                # Check if red spaceship has fired
                if event.key == pygame.K_RCTRL and len(self.red_bullets) < MAX_BULLETS:
                    self.bullet = pygame.Rect(self.red.x, self.red.y + self.red.height//2, 10, 5)
                    self.red_bullets.append(self.bullet)
                    BULLET_FIRE_SOUND.play()
                
        # Get which keys have been pressed 
        keys_pressed = pygame.key.get_pressed()
        # Yellow spaceship
        if keys_pressed[pygame.K_a] and self.yellow.x - VEL > 0:  # LEFT
            self.yellow.x -= VEL
        if keys_pressed[pygame.K_d] and self.yellow.x + VEL + self.yellow.width < BORDER.x:  # RIGHT
            self.action = Action.LEFT
            self.yellow.x += VEL
        if keys_pressed[pygame.K_w] and self.yellow.y - VEL > 0:  # UP
            self.yellow.y -= VEL
        if keys_pressed[pygame.K_s] and self.yellow.y + VEL + self.yellow.height < HEIGHT - 15:  # DOWN
            self.yellow.y += VEL

        # Red spaceship
        if keys_pressed[pygame.K_j] and self.red.x - VEL > BORDER.x + BORDER.width:  # LEFT
            self.red.x -= VEL
        if keys_pressed[pygame.K_l] and self.red.x + VEL + red.width < WIDTH:  # RIGHT
            self.red.x += VEL
        if keys_pressed[pygame.K_i] and self.red.y - VEL > 0:  # UP
            self.red.y -= VEL
        if keys_pressed[pygame.K_k] and self.red.y + VEL + self.red.height < HEIGHT - 15:  # DOWN
            self.red.y += VEL
    
        # Handle bullets and check if a spaceship is hit          
        self.handle_bullets()

        # 3. Check if game is over
        game_over = False     
        if self.red_health <= 0:
            game_over = True
            self.winner_text = "Yellow Wins!"
            self.winner_colour = YELLOW
            self.draw_winner()
            return game_over, self.yellow_health, self.red_health

        if self.yellow_health <= 0:
            game_over = True
            self.winner_text = "Red Wins!"
            self.winner_colour = RED
            self.draw_winner()
            return game_over, self.yellow_health, self.red_health
        
        # 4. Update UI and Clock 
        self.draw_window()
        self.clock.tick(FPS)
        return game_over, self.yellow_health, self.red_health
        
    def handle_bullets(self):
        # Handle yellow bullets
        for self.bullet in self.yellow_bullets:
            self.bullet.x += BULLET_VEL
            if self.red.colliderect(self.bullet):
                self.red_health -= 1
                self.yellow_bullets.remove(self.bullet)
                BULLET_HIT_SOUND.play()
            elif self.bullet.x > WIDTH:
                self.yellow_bullets.remove(self.bullet)
        
        # Handle red bullets
        for self.bullet in self.red_bullets:
            self.bullet.x -= BULLET_VEL
            if self.yellow.colliderect(self.bullet):
                self.yellow_health -= 1
                self.red_bullets.remove(self.bullet)
                BULLET_HIT_SOUND.play()
            elif self.bullet.x < 0:
                self.red_bullets.remove(self.bullet)
            
    # Render
    def draw_window(self):
        WIN.blit(SPACE, (0, 0))
        pygame.draw.rect(WIN, PURPLE, BORDER)

        self.red_health_text = HEALTH_FONT.render(str(self.red_health), 1, RED)
        self.yellow_health_text = HEALTH_FONT.render(str(self.yellow_health), 1, RED)
        
        # Draw Spaceship health 
        WIN.blit(HEART, (WIDTH - 100, 17))
        WIN.blit(self.red_health_text, (WIDTH - 50, 10))
        WIN.blit(HEART, (10, 17))
        WIN.blit(self.yellow_health_text, (60, 10))
        
        # Draw Spaceships
        WIN.blit(YELLOW_SPACESHIP, (self.yellow.x, self.yellow.y))
        WIN.blit(RED_SPACESHIP, (self.red.x, self.red.y))

        for self.bullet in self.red_bullets:
            pygame.draw.rect(WIN, RED, self.bullet)

        for self.bullet in self.yellow_bullets:
            pygame.draw.rect(WIN, YELLOW, self.bullet)

        pygame.display.update()   
    
    def draw_winner(self):
        draw_text = WINNER_FONT.render(self.winner_text, 1, self.winner_colour)
        WIN.blit(draw_text, (WIDTH/2 - draw_text.get_width()/2, HEIGHT/2 - draw_text.get_height()/2))
        pygame.display.update()
        pygame.time.delay(2500)

In [ ]:
if __name__ == '__main__':
    game = SpaceShooterEnv()
    game_number = 0
    
    #game loop
    while True:
        game_over, yellow_health, red_health = game.step()
        
        if game_over == True:
            game.reset() 
            game_number += 1
            print("--------- Game number", game_number, "---------")
            print("Yellow health:", yellow_health)
            print("Red health:", red_health)